# Automatic credibility assessment of popular science articles available online

## Module 1: Topic modelling

In [2]:
# Load dependencies

import numpy as np
import nltk
import re
import os
import codecs

from morfeusz import analyse
import gensim
import gensim.corpora as corpora


import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import numpy as np

# Load polish tokenizer from NLTK

In [2]:
polish_tokenizer = nltk.data.load('tokenizers/punkt/polish.pickle')

# Load raw article texts

In [6]:
def raw_text_list(path_to_articles='articles'):
    articles = []
    labels = []

    for filename in os.listdir(path_to_articles):
        article = ""

        if not filename.startswith('.') and filename.endswith(".txt"): 
            # print(os.path.join(directory, filename))
            f = open(path_to_articles + "/" + filename)
            lines = f.readlines()
            article = " ".join(lines)
            labels.append(filename)
            articles.append(article)

    return articles, labels

# print first elements of 2 parallell lists from 2xN array (where N is the number of articles)

articles, labels = raw_text_list()

for i in range(0,7):
    print(articles[i][0:100] + '...')
    print(labels[i] + '\n')

 ﻿Loratadyna jest lekiem wprowadzonym na rynek w 1993 r. pod nazwą Claritine przez koncern Schering-...
29.txt

 ﻿Mięśniaki – najczęstsze łagodne guzy macicy Mięśniaki są to guzy łagodne, wywodzące się z włókien ...
15.txt

 Co roku zaleca się ludziom przyjmowanie szczepionki przeciw grypie, co ma oszczędzić im powtarzając...
114.txt

 Jednym z podstawowych elementów edukacji moich dietetycznych pacjentów jest nauczenie interpretowan...
100.txt

 Medycyna ludowa od dawna wykorzystuje lecznicze właściwości pestek dyni.
  Używa się ich, zarówno u...
128.txt

 Żywienie Optymalne zapewnia wszystkim, którzy je stosują, eliminowanie przyczyn chorób.
  Jest to j...
129.txt

 Ludzki organizm, głównie w wątrobie, sam wytwarza cholesterol, bo jest on nam potrzebny.
  W zdrowo...
101.txt



# Tokenization & stemming

In [7]:
def base_word(word):
    return analyse(word)[0][0][1]

def extract_words_from_string(s):
    return re.findall(re.compile('\w+'), s.lower())

def tokenize_and_stem(article):
    tokens = [word for sent in polish_tokenizer.tokenize(article) for word in nltk.tokenize.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)

    stems = []
    for t in filtered_tokens:
        try:
            stem = base_word(t)
        except KeyError as e:
            stem = t
        stems.append(stem)

    return stems

def article_vocabulary_stemmed(article):
    '''
        'article' is of type string
    '''

    word_list = extract_words_from_string(article)
    article_vocabulary = []

    for w in word_list:
        try:
            a = base_word(w)
            article_vocabulary.append(a)
        except KeyError as e:
            pass

    return article_vocabulary

def data_lemmatized(articles):
    articles_lemmatized = []
    for article in articles:
        articles_lemmatized.append(article_vocabulary_stemmed(article))
        
    return articles_lemmatized

articles_lemmatized = data_lemmatized(articles)
articles_lemmatized[0]

['loratadyna',
 'być',
 'lek',
 'wprowadzić',
 'na',
 'rynek',
 'w',
 'r',
 'pod',
 'nazwać',
 'claritine',
 'przez',
 'koncern',
 'schering',
 'plough',
 'europ',
 'ten',
 'popularny',
 'farmaceutyk',
 'o',
 'działanie',
 'przeciwalergiczny',
 'oraz',
 'przeciwhistaminowy',
 'w',
 'jak',
 'sposób',
 'dziać',
 'loratadyna',
 'loratadyna',
 'wykazywać',
 'selektywny',
 'antagonistyczny',
 'działanie',
 'na',
 'obwodowy',
 'receptor',
 'h1',
 'lek',
 'należeć',
 'stosować',
 'doustnie',
 'ponieważ',
 'dobrze',
 'oraz',
 'szybko',
 'wchłaniać',
 'się',
 'z',
 'przewód',
 'pokarmowy',
 'podczas',
 'pierwsze',
 'przejście',
 'przez',
 'wątroba',
 'być',
 'niemal',
 'całkowicie',
 'metabolizować',
 'przy',
 'udział',
 'cyp2d6',
 'oraz',
 'cyp3a4',
 'desloratadyna',
 'być',
 'główny',
 'metabolit',
 'czynny',
 'farmakologicznie',
 'odpowiedzialny',
 'za',
 'odpowiedni',
 'działanie',
 'lek',
 'wysoki',
 'stężenie',
 'loratadyny',
 'we',
 'krew',
 'obserwować',
 'się',
 'po',
 'upływ',
 'oko',

# Get the article vocabulary

In [4]:
articles_lemmatized = data_lemmatized(articles)
articles_lemmatized[:1]

NameError: name 'data_lemmatized' is not defined

# Term document frequency

In [6]:
id2word = corpora.Dictionary(articles_lemmatized)

articles_lem = articles_lemmatized

corpus = [id2word.doc2bow(article) for article in articles_lem]

import gensim.downloader as api
from gensim.sklearn_api import TfIdfTransformer
import pickle

model = TfIdfTransformer(id2word=id2word) # -- use for the first time
tfidf_corpus = model.fit_transform(corpus)
# model = picle.load(open('tfidf_medical_corpus.p', 'rb')) 
# pickle.dump(model, open('tfidf_medical_corpus.p', 'wb')) # -- use if model has already been pickled


# Use the model to show retrieved context

In [3]:
n = 243 # first document's ID
imp = 5 # amount of the most important words for the document

# print(tfidf_corpus[n])
indexes, scores = [list(a) for a in zip(*tfidf_corpus[n])]

import numpy as np

np_scores = np.array(scores) # transform scores to numpy array to use the sorting function
idxs_most_important_words = np_scores.argsort()[-imp:] 

context = ''
for idx in reversed(idxs_most_important_words):
    context += (id2word[indexes[idx]] + ' ')
    
print(context)
print('\n')
for i in range (n-5, n+5):
    print(articles[i][0:240] + '...')

NameError: name 'tfidf_corpus' is not defined

## Module 2: WordNet & Hyperonymy (meaning aggregates)

### plWordNet XML parser

In [3]:
import xml.etree.ElementTree as ET
import pickle

### Load WNet XML to the parser
tree = pickle.load(open('plwntree.dump', 'rb'))

# For the first run -->

# tree = ET.parse('plwordnet-4.0.xml')
# pickle.dump(tree, open('plwntree.dump', 'w'))


### Load lexical unit <-> sysnet mapping in Python readable format
synunitmap = pickle.load(open('synunitmap.dump', 'rb'))

# for the first run -->

# synunitmap = synsetUnitMapping() # for the first run
# pickle.dump(synunitmap, open('synunitmap.dump', 'wb'))


root = tree.getroot()

def synsetUnitMapping(root=root):
    synsets = dict()
    for child in root.iter():
        if child.tag == "synset":
            synset_id = child.attrib['id']
            unit_list = list(child.findall("unit-id"))
            synsets[synset_id] = [uid.text for uid in unit_list]
            
    return synsets

def printNFirstMappings(n=100, synunitmap=synunitmap):
    from heapq import nlargest

    for key in nlargest(10, synunitmap, key=synunitmap.get):
        print(key, synunitmap[key])
    
def wordFromId(word_id):
    for child in root.findall('lexical-unit'):
        if child.attrib['id'] == word_id:
            return child.attrib['name']
    raise ValueError("No such lexical unit ID in plWordNet")
                        
    
def idFromWord(word):
    for child in root.findall('lexical-unit'):
        if child.attrib['name'] == word:
            return child.attrib['id']
    raise ValueError("No such lexical unit in plWordNet")
    

def synsetsForWord(word='', synunitmap = synunitmap, word_id='', tagtype='name'):
    ''' 
    Find synset for a word (given by id or name).
    The loop iterates through synset - lex. unit mapping.
    '''
    if tagtype == 'name':
        word_id = idFromWord(word)
    
    synsets = []
    for s in synunitmap.keys():
        if word_id in synunitmap[s]:
            synsets.append(s)
    
    return synsets


def getHyponymsFromWord(word, synunitmap = synunitmap):
    synsets_for_word = synsetsForWord(word=word)
    hypo_synsets = []
    
    for child in root.findall('synsetrelations'):
        if child.attrib['relation'] in ['11', '211'] and child.attrib['parent'] in synsets_for_word:
            hypo_synsets.append(child.attrib['child'])
    
    hyponyms = []
    for s in hypo_synsets:
        hyponyms = list(set().union(hyponyms,synunitmap[s]))
    
    return [wordFromId(word_id) for word_id in hyponyms]


In [106]:
getHyponymsFromWord('objaw chorobowy')

['ostry brzuch',
 'hiperprolaktynemia',
 'anhedonia',
 'zespół suchego oka',
 'abulia',
 'leukocytoza',
 'kulawizna',
 'upośledzenie',
 'ambitendencja',
 'objaw Aschaffenburga',
 'biegunka',
 'zmiana chorobowa',
 'alalia',
 'swędzenie',
 'wytrzeszcz złośliwy',
 'eozynofilia',
 'nosicielstwo',
 'autoskopia',
 'perseweracja',
 'wstrząs',
 'puchlina wodna',
 'rozwolnienie',
 'bielmo',
 'azoospermia',
 'entropium',
 'mioza',
 'niedokrwienie',
 'kwasica',
 'leukokoria',
 'wzdęcie',
 'nadciśnienie',
 'męczliwość',
 'bakteriomocz',
 'słowotok',
 'aura',
 'bezmocz',
 'ektropion',
 'odrętwienie',
 'paroksyzm',
 'światłowstręt',
 'inkoherencja',
 'wodogłowie',
 'afonia',
 'koprolalia',
 'ruptura',
 'gorączka',
 'pasma Meesa',
 'małopłytkowość',
 'schizofazja',
 'skąpomocz',
 'stawy Charcota',
 'leukopenia',
 'zawał',
 'ciągotka',
 'bakteriuria',
 'chloroza',
 'stan spastyczny',
 'objaw zwiastunowy',
 'chrypa',
 'majaczenie',
 'palpitacja',
 'bezwład',
 'hipertrofia',
 'oftalmoplegia',
 'skleroza

## Load sentences and visualize then in a window

### Load

In [107]:
'''
    Load sentences as a list of dicts:
    [
        ...,
        { 'body': 'sentence body',
          'contextual_rate': 1,
          'uncontextual_rate': -1,
          'article_id': 10,
          'position_in_article': 3},
        ...
    ]
'''
sentences = pickle.load(open('sentences_dict.dump', 'rb'))
sentences[1100]

{'article_id': '16\n',
 'body': ' A jako stan ostrzegawczy sytuację, gdy poziom glukozy na czczo jest podwyższony, ale nie przekracza 7mmol/l.',
 'contextual_rate': '2',
 'position_in_article': 53,
 'uncontextual_rate': '2'}

### Visualize

In [96]:
LEFT_WINDOW_SIZE = 2
RIGHT_WINDOW_SIZE = 1

class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

def is_inassessible(sent):
    return sent['uncontextual_rate'] == '-1'
        
def get_article(art_id, padding=3, sentences=sentences):
    art = []
    for sent in sentences:
        if sent['article_id'] == art_id:
            art.append(sent['body'])
            
    return ['']*padding + art + ['']*padding
                
def print_sent_with_window(sent, lw=3, rw=1, sentences=sentences):
    pos = sent['position_in_article']
    padding = 3
    begin = pos - lw
    end = pos + rw
    art = get_article(sent['article_id'])
    
    res = [color.BOLD + s + color.END if art.index(s) + 1 == pos else s for s in art[begin:end]]
    for r in res:
        print(r)
        

 ﻿Zawał serca jest jedną z najczęstszych przyczyn zgonów występująca u Polaków oraz innych mieszkańców Europy.
 Wiele osób nie zdaje sobie sprawy z tego, że w zwalczaniu skutków tej choroby największe znaczenie ma czas.
 Im szybciej zostanie udzielona fachowa pomoc pacjentowi, tym większe są szanse na powrót do pełni zdrowia i zmniejszenie skutków ubocznych martwicy komórek sercowych oraz ograniczenie rozległości zmian zawałowych w obrębie organu.


### Get inassessable sentences

In [116]:
def get_inassessible_sentences(sentences=sentences):
    inass = []
    for sent in sentences:
        if is_inassessible(sent):
            inass.append(sent)
    return inass

inassessible = get_inassessible_sentences()

START = 810

for sent in inassessible[START:START + 7]:
    print_sent_with_window(sent)
    print('\n')

 Podobno w wielu przypadkach terapia z ich zastosowaniem daje całkiem niezłe rezultaty.
 Niestety w razie nieprzestrzegania standardów bezpieczeństwa może stwarzać poważne zagrożenie dla zdrowia.
 Czy warto korzystać z usług hirudoterapeutów? A jeśli jak, to jakie środki ostrożności warto zachować? Hirudoterapia – tak nazywa się metoda leczenia z wykorzystaniem pijawki lekarskiej, "żyjątka" wyposażonego w trzy szczęki, z których każda zawiera od 80 do 90 chitynowych zębów.
 Nie jest to wynalazek współczesnych "szamanów", ale praktyka znana od starożytności.


 Czy warto korzystać z usług hirudoterapeutów? A jeśli jak, to jakie środki ostrożności warto zachować? Hirudoterapia – tak nazywa się metoda leczenia z wykorzystaniem pijawki lekarskiej, "żyjątka" wyposażonego w trzy szczęki, z których każda zawiera od 80 do 90 chitynowych zębów.
 Nie jest to wynalazek współczesnych "szamanów", ale praktyka znana od starożytności.
 Pijawki, zwierzęta należące do gromady siodełkowców – tej samej, 